In [1]:
import ZODB, ZODB.FileStorage
import persistent
from BTrees.OOBTree import TreeSet, OOBTree
import transaction
from word_anal import define

In [2]:
storage = ZODB.FileStorage.FileStorage('database.fs')
db = ZODB.DB(storage)
conn = db.open()
root = conn.root()

Ignoring index for /home/fn/Projects/solid-spork/db/database.fs


In [3]:
class Dictionary(persistent.Persistent):
    def __init__(self, title):
        self.title = title
        self.words = OOBTree()
    
    def add_word(self, word):
        self.words.update({word:Word(word)})
        
    def __repr__(self):
        return self.title

In [79]:
class Word(persistent.Persistent):
    def __init__(self, name):
        self.name = name
        self.not_defined = None
        
        # list of different pronunciations
        self.pronunciation = persistent.list.PersistentList()
        
        # synonyms and their exlpanations
        self.syns = OOBTree()
        
        if self.analyse() == 0:
            self.not_defined = True
        else:
            self.not_defined = False
        
    def analyse(self):
        a = define(self.name)
        if a == None:
            return 0
        self.pronunciation = a['pronunciation']
        self.syns = OOBTree(a['syns'])
    
    def __repr__(self):
        if self.not_defined == True:
            return self.name
        elif self.not_defined == False:
            # return the word and its synonyms
            return str(self.name + ': ' + ', '.join(set([g.split('.')[0] for g in self.syns.keys()])))
        else:
            return self.name + ' (analysis not yet run)'
        
    def __eq__(self, other):
        if self.name == other.name:
            return True
        else return False

In [80]:
dictio = Dictionary('naziv')

In [81]:
dictio.add_word('human')

In [82]:
dictio.add_word('bla')

In [83]:
list(dictio.words.values('bla'))

[bla, human: human, homo]

In [77]:
db.close()

In [78]:
conn.close() 